# MASK R-CNN using pytorch, torchvision

- object detection, instance segmentation
- Ref: https://github.com/spmallick/learnopencv/blob/master/PyTorch-Mask-RCNN/PyTorch_Mask_RCNN.ipynb

Install cv2 in venv_torch2

```bash
source ~/Workspace/venv_torch2/bin/activate
pip3 install opencv-python
pip3 freeze > ~/Workpace/venv_torch2/requirements
deactivate
```

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as T
import torchvision
import torch
import numpy as np
import cv2
import random
import time
import os

# These are the classes that are available in the COCO-Dataset
COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]
# Regarding N/A (10 missing categories) refer to https://tech.amikelive.com/node-718/what-object-categories-labels-are-in-coco-dataset/
# 80 categories are released in 2014 and 2017 and used
# FYI, the original paper mentioned 91 categories (10 N/A's, __background__)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

## Download Model

In [ ]:
# get the pretrained model from torchvision.models
# Note: pretrained=True will get the pretrained weights for the model.
# model.eval() mode change to use the model for inference (from train mode)
#model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights=True) # deprecated, userwarning
model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights='MaskRCNN_ResNet50_FPN_Weights.DEFAULT')
model.to(device)
model.eval()

In [ ]:
def random_colour_masks(image):
    """
    random_colour_masks
    parameters:
      - image - predicted masks
    method:
      - the masks of each predicted object is given random colour for visualization
    """
    colours = [[0, 255, 0],[0, 0, 255],[255, 0, 0],[0, 255, 255],[255, 255, 0],[255, 0, 255],[80, 70, 180],[250, 80, 190],[245, 145, 50],[70, 150, 250],[50, 190, 190]]
    r = np.zeros_like(image).astype(np.uint8)
    g = np.zeros_like(image).astype(np.uint8)
    b = np.zeros_like(image).astype(np.uint8)
    r[image == 1], g[image == 1], b[image == 1] = colours[random.randrange(0,10)]
    coloured_mask = np.stack([r, g, b], axis=2)
    return coloured_mask

def get_prediction(img_path, threshold):
    """
    get_prediction
    parameters:
      - img_path - path of the input image
    method:
      - Image is obtained from the image path
      - the image is converted to image tensor using PyTorch's Transforms
      - image is passed through the model to get the predictions
      - masks, classes and bounding boxes are obtained from the model and soft masks are made binary(0 or 1) on masks
        ie: eg. segment of cat is made 1 and rest of the image is made 0

    """
    img = Image.open(img_path) # load an image from a file path 
    transform = T.Compose([T.ToTensor()]) # define transform to tensor
    img = transform(img).to(device) # image to tensor image
    pred = model([img])  # model prediction
    pred_score = list(pred[0]['scores'].detach().cpu().numpy()) # scores :tensor to numpy, detach() is required to remove grad info 
    pred_t = [pred_score.index(x) for x in pred_score if x>threshold][-1]
    # tensor.squeeze(): remove dimension 1, # ex. [3, 1, 20, 128] -> [3, 20, 128]
    masks = (pred[0]['masks']>0.5).squeeze().detach().cpu().numpy() # gpu to cpu, to numpy
    pred_class = [COCO_INSTANCE_CATEGORY_NAMES[i] for i in list(pred[0]['labels'].detach().cpu().numpy())]
    pred_boxes = [[(i[0], i[1]), (i[2], i[3])] for i in list(pred[0]['boxes'].detach().cpu().numpy())]
    masks = masks[:pred_t+1]
    pred_boxes = pred_boxes[:pred_t+1]
    pred_class = pred_class[:pred_t+1]
    return masks, pred_boxes, pred_class

def instance_segmentation(img_path, threshold=0.5, rect_th=3, text_size=3, text_th=3):
    """
    instance_segmentation
    parameters:
      - img_path - path to input image
    method:
      - prediction is obtained by get_prediction
      - each mask is given random color
      - each mask is added to the image in the ration 1:0.8 with opencv
      - final output is displayed
    """
    masks, boxes, pred_cls = get_prediction(img_path, threshold)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    for i in range(len(masks)):
        rgb_mask = random_colour_masks(masks[i])
        img = cv2.addWeighted(img, 1, rgb_mask, 0.5, 0)
        box_i1 = (int(boxes[i][0][0]), int(boxes[i][0][1]))
        box_i2 = (int(boxes[i][1][0]), int(boxes[i][1][1]))
        cv2.rectangle(img, box_i1, box_i2,color=(0, 255, 0), thickness=rect_th)
        cv2.putText(img,pred_cls[i], box_i1, cv2.FONT_HERSHEY_SIMPLEX, text_size, (0,255,0),thickness=text_th)
    plt.figure(figsize=(20,30))
    plt.imshow(img)
    plt.xticks([])
    plt.yticks([])
    plt.show()


In [ ]:
# We will use the following colors to fill the pixels
colours = [[0, 255, 0],
           [0, 0, 255],
           [255, 0, 0],
           [0, 255, 255],
           [255, 255, 0],
           [255, 0, 255],
           [80, 70, 180],
           [250, 80, 190],
           [245, 145, 50],
           [70, 150, 250],
           [50, 190, 190]]

In [ ]:
# images in this path to be tested
image_path = '../images/'

In [ ]:
# Plot the image 

img = Image.open(image_path+'./mrcnn_person.jpg')
plt.imshow(img)
plt.axis('off')
plt.title(f'image size:{img.size}')
plt.show()

In [ ]:
# Running inference on the image
# T.Compose: can compose of a series of preprocessing, for example, resize, random horizonal flip, totensor, normalize
# here, tranform to tensor for model input
transform = T.Compose([T.ToTensor()])
img_tensor = transform(img).to(device) 
# predict from input image tensor in array
pred = model([img_tensor])

In [ ]:
# Lets look at what the `pred`, the result, looks like.
# `pred` is a list of dictionaries, since we had passed a single image, we will get a single-item list
pred[0]

In [ ]:
# We will keep only the pixels with values  greater than 0.5 as 1, and set the rest to 0.
masks = (pred[0]['masks']>0.5).squeeze().detach().cpu().numpy()
masks.shape

In [ ]:
# Let's plot the mask for the `person` class since the 0th mask belongs to `person`
plt.imshow(masks, cmap='gray')
plt.show()

In [ ]:
# Let's color the `person` mask using the `random_colour_masks` function
mask1 = random_colour_masks(masks)
plt.imshow(mask1)
plt.show()

In [ ]:
# Let's blend the original and the masked image and plot it.
blend_img = cv2.addWeighted(np.asarray(img), 0.5, mask1, 0.5, 0)

plt.imshow(blend_img)
plt.show()

In [ ]:
instance_segmentation(image_path+'./mrcnn_standing_people.jpg', 0.75)

In [ ]:
instance_segmentation(image_path+'./mrcnn_cars.jpg', 0.9, rect_th=5, text_size=5, text_th=5)

In [ ]:
instance_segmentation(image_path+'./mrcnn_traffic.jpg', 0.6, rect_th=2, text_size=2, text_th=2)

In [ ]:
instance_segmentation(image_path+'./mrcnn_birds.jpg', 0.9)  

In [ ]:
instance_segmentation(image_path+'./mrcnn_people.jpg', 0.8, rect_th=1, text_size=1, text_th=1)    

In [ ]:
instance_segmentation(image_path+'./mrcnn_playing.jpg', 0.8, rect_th=6, text_size=6, text_th=6)   

In [ ]:
instance_segmentation(image_path+'./mrcnn_cat_dog.jpg', 0.95, rect_th=5, text_size=5, text_th=5)    

In [ ]:
instance_segmentation(image_path+'./mrcnn_elephants.jpg', 0.9, rect_th=6, text_size=6, text_th=6)    

In [ ]:
instance_segmentation(image_path+'./mrcnn_baby_teddy.jpg', 0.95, rect_th=6, text_size=6, text_th=6)    

## Comparing the inference time of model in CPU & GPU

In [ ]:
def check_inference_time(image_path, gpu=False):
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
    model.eval()
    img = Image.open(image_path)
    transform = T.Compose([T.ToTensor()])
    img = transform(img)
    if gpu:
        model.cuda()
        img = img.cuda()
    else:
        model.cpu()
        img = img.cpu()
    start_time = time.time()
    pred = model([img])
    end_time = time.time()
    return end_time-start_time

In [ ]:
def check_inference_time(image_path, gpu=False):
    #model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True) # user warning
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights='MaskRCNN_ResNet50_FPN_Weights.DEFAULT')
    model.eval()
    img = Image.open(image_path)
    transform = T.Compose([T.ToTensor()])
    img = transform(img)
    if gpu:
        model.cuda()
        img = img.cuda()
    else:
        model.cpu()
        img = img.cpu()
    start_time = time.time()
    pred = model([img])
    end_time = time.time()
    return end_time-start_time

In [ ]:
# Let's run inference on all the downloaded images and average their inference time 
img_paths = [path for path in os.listdir(image_path) if (path.split(".")[-1].lower() in ["jpeg", "jpg"]) and path.startswith('mrcnn_') ]

gpu_time = sum([check_inference_time(image_path+img_path, gpu=True) for img_path in img_paths])/len(img_paths)
cpu_time = sum([check_inference_time(image_path+img_path, gpu=False) for img_path in img_paths])/len(img_paths)

print('\n\nAverage Time take by the model with GPU = {}s\nAverage Time take by the model with CPU = {}s'.format(gpu_time, cpu_time))

In [ ]:
cpu_time/gpu_time

In [ ]:
# minor comments
# this category names include "N/A"

# len(COCO_INSTANCE_CATEGORY_NAMES)
# nas = [ c for c in COCO_INSTANCE_CATEGORY_NAMES if c == "N/A"]
# len(nas), nas